In [1]:
import numpy as np
import itertools
from typing import List, Tuple

In [2]:
def set_diag_vector(diagonal_length: int) -> List[int]:
    if diagonal_length <= 1:
        raise ValueError("diagonal_length must be positive and greater than one.")
    if diagonal_length == 1:
         return [1]
    return [1] + [0] * (diagonal_length-1)


In [3]:
def set_on_off_measurement_one_outcome(outcome: int, cutoff_dim: int, diag_vector: List[int] = None) -> np.ndarray:
    if outcome != 0 and outcome != 1:
        raise ValueError("outcome must be either 0 or 1")
    if diag_vector is None:
        diag_vector = set_diag_vector(diagonal_length=cutoff_dim+1)
    if outcome == 0:
        return np.diag(diag_vector)
    return np.identity(n=cutoff_dim+1) - np.diag(diag_vector)
    

In [4]:
set_on_off_measurement_one_outcome(outcome=1, cutoff_dim=2)

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [496]:
def generate_all_outcomes(modes=int) -> List[Tuple[int, ...]]:
    options = [0, 1]
    return [*itertools.product(options, repeat=modes)]


In [497]:
generate_all_outcomes(modes=2)

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [7]:
def compute_kronecker_matrices(matrices: List[np.ndarray]) -> np.ndarray:
    if len(matrices) < 2:
        raise ValueError("The number of matrices must be at least 2")
    if len(matrices) == 2:
        return np.kron(matrices[0], matrices[1])
    last_matrix = matrices.pop()
    return np.kron(compute_kronecker_matrices(matrices), last_matrix)


In [8]:
def generate_on_off_measurement_one_outcome(one_outcome: Tuple[int], cutoff_dim: int, diag_vector: List[int] = None) -> np.ndarray:
    if diag_vector is None:
        diag_vector = set_diag_vector(diagonal_length=cutoff_dim + 1)
    one_mode_measurements = [set_on_off_measurement_one_outcome(
        outcome=outcome_i, cutoff_dim=cutoff_dim, diag_vector=diag_vector) for outcome_i in one_outcome]
    return compute_kronecker_matrices(matrices=one_mode_measurements)

In [9]:
def generate_all_on_off_measurements(modes: int, cutoff_dim: int) -> List[np.ndarray]:
    outcomes = generate_all_outcomes(modes=modes)
    diag_vector = set_diag_vector(diagonal_length=cutoff_dim + 1)
    
    return [generate_on_off_measurement_one_outcome(one_outcome=outcome, cutoff_dim=cutoff_dim, diag_vector=diag_vector) 
            for outcome in outcomes]

In [10]:
generate_all_on_off_measurements(modes=2, cutoff_dim=1)

[array([[1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]),
 array([[0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.]]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 1.]])]

In [11]:
generate_all_on_off_measurements(modes=2, cutoff_dim=2)


[array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0.

In [12]:
import strawberryfields as sf
from strawberryfields.backends import BaseFockState

In [452]:
alphas = [0.5]
betas = [0.3]
modes = 3
cutoff_dim = 3

In [453]:
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": cutoff_dim})
prog = sf.Program(modes)

par_alpha = prog.params("alpha")
par_beta = prog.params("beta")

with prog.context as q:
    # sf.ops.Vacuum() | q[0]
    # sf.ops.Vacuum() | q[1]
    sf.ops.Dgate(par_alpha) | q[0]
    sf.ops.Dgate(par_beta) | q[1]
    sf.ops.BSgate() | (q[0], q[1])
    sf.ops.BSgate() | (q[1], q[2])


In [454]:
result = eng.run(prog, args={
    "alpha": alphas[0],
    "beta": betas[0],
})

In [451]:
result.state.data.shape

TensorShape([10, 10, 10])

In [403]:
result.state.data

array([[[[0.+0.j, 0.+0.j],
         [0.+0.j, 1.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]]],


       [[[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]]]])

In [419]:
sum(sum(result.state.all_fock_probs()))

0.9862470886252108

In [404]:
result.state.fock_prob([0,1])

1.0

In [398]:
result.state.dm().shape

(2, 2, 2, 2)

In [399]:
result.state.dm()


array([[[[1.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]]],


       [[[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]],

        [[0.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]]]])

In [79]:
result.state.all_fock_probs().shape

(7, 7)

In [80]:
result.state.all_fock_probs()

array([[5.59898367e-01, 2.79949183e-01, 6.99872958e-02, 1.16645493e-02,
        1.45806866e-03, 1.45806866e-04, 1.21505722e-05],
       [4.47918693e-02, 2.23959347e-02, 5.59898367e-03, 9.33163944e-04,
        1.16645493e-04, 1.16645493e-05, 6.13833400e-07],
       [1.79167477e-03, 8.95837387e-04, 2.23959347e-04, 3.73265578e-05,
        4.66581972e-06, 1.09262933e-07, 1.23267605e-11],
       [4.77779939e-05, 2.38889970e-05, 5.97224924e-06, 9.95374874e-07,
        9.99080760e-09, 5.53559913e-09, 1.99585807e-10],
       [9.55559879e-07, 4.77779939e-07, 1.19444985e-07, 9.85995461e-08,
        3.91584308e-09, 1.57618751e-12, 1.62074802e-12],
       [1.52889581e-08, 7.64447903e-09, 9.42090477e-08, 9.45222172e-12,
        1.01795139e-10, 2.29784692e-12, 8.44458742e-15],
       [2.03852774e-10, 3.74154971e-08, 3.18870995e-09, 6.27435524e-11,
        2.21359253e-13, 5.27786713e-14, 7.75846469e-16]])

In [424]:
sum(sum(result.state.all_fock_probs()))


0.9999999999997908

In [82]:
result.state.fock_prob([0,2])

0.06998729582067524

In [231]:
def get_index_shape_from_outcome_one_mode(outcome_one_mode: int, cutoff_dim: int) -> Tuple[int, int]:
    if outcome_one_mode != 0 and outcome_one_mode != 1:
        raise ValueError("Outcome mode must be either 0 or 1")
    if outcome_one_mode == 0:
        return (0, 1)
    return (1, cutoff_dim)


In [235]:
def get_index_shape_from_outcome(outcome: Tuple[int], cutoff_dim: int) -> Tuple[int, int, int, int]:
    result = ()
    for outcome_one_mode in outcome:
        result += get_index_shape_from_outcome_one_mode(outcome_one_mode=outcome_one_mode, cutoff_dim=cutoff_dim)
    return result


In [236]:
get_index_shape_from_outcome(outcome=(1,0), cutoff_dim=3)

(1, 3, 0, 1)

In [257]:
def generate_measurement_matrix_one_outcome(outcome: Tuple[int], cutoff_dim: int, zeros_matrix: np.ndarray) -> np.ndarray:

    ones_matrix = np.ones((1,1), dtype=np.int32)
    indices = get_index_shape_from_outcome(outcome=outcome, cutoff_dim=cutoff_dim)
    print(indices)
    final_matrix = zeros_matrix.copy()
    if len(indices) < 2 or len(indices) > 14:
        raise ValueError("modes not supported. Only from 1 to 7 modes supported.")
    if len(indices) == 2:
        final_matrix[indices[0]:indices[1]] = ones_matrix
    if len(indices) == 4:
        final_matrix[indices[0]:indices[1], indices[2]:indices[3]] = ones_matrix
    if len(indices) == 6:
        final_matrix[indices[0]:indices[1], indices[2]:indices[3], indices[4]:indices[5]] = ones_matrix
    if len(indices) == 8:
        final_matrix[indices[0]:indices[1], indices[2]:indices[3],
                     indices[4]:indices[5], indices[6]:indices[7]] = ones_matrix
    if len(indices) == 10:
        final_matrix[indices[0]:indices[1], indices[2]:indices[3],
                     indices[4]:indices[5], indices[6]:indices[7], indices[8]:indices[9]] = ones_matrix
    if len(indices) == 12:
        final_matrix[indices[0]:indices[1], indices[2]:indices[3],
                     indices[4]:indices[5], indices[6]:indices[7],
                     indices[8]:indices[9], indices[10]:indices[11]] = ones_matrix
    if len(indices) == 14:
        final_matrix[indices[0]:indices[1], indices[2]:indices[3],
                     indices[4]:indices[5], indices[6]:indices[7],
                     indices[8]:indices[9], indices[10]:indices[11], indices[12]:indices[13]] = ones_matrix
    
    return final_matrix


In [339]:
def generate_measurement_matrices(num_modes: int, cutoff_dim: int) -> List[np.ndarray]:
    matrix_shape = [cutoff_dim] * num_modes
    zeros_matrix = np.zeros(matrix_shape, dtype=np.float32)
    outcomes = generate_all_outcomes(modes=num_modes)
    print(outcomes)
    return [generate_measurement_matrix_one_outcome(outcome=outcome, cutoff_dim=cutoff_dim, zeros_matrix=zeros_matrix) 
            for outcome in outcomes]
    

In [429]:
cutoff_dim = 3
num_modes = 3
measurement_matrices = generate_measurement_matrices(num_modes=num_modes, cutoff_dim=cutoff_dim)


[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
(0, 1, 0, 1, 0, 1)
(0, 1, 0, 1, 1, 3)
(0, 1, 1, 3, 0, 1)
(0, 1, 1, 3, 1, 3)
(1, 3, 0, 1, 0, 1)
(1, 3, 0, 1, 1, 3)
(1, 3, 1, 3, 0, 1)
(1, 3, 1, 3, 1, 3)


In [447]:
measurement_matrices[7]

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1.]

In [438]:
all_focks = result.state.all_fock_probs()


In [443]:
all_focks = result.state.all_fock_probs()
measurement_matrices = generate_measurement_matrices(num_modes=num_modes, cutoff_dim=cutoff_dim)


[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
(0, 1, 0, 1, 0, 1)
(0, 1, 0, 1, 1, 10)
(0, 1, 1, 10, 0, 1)
(0, 1, 1, 10, 1, 10)
(1, 10, 0, 1, 0, 1)
(1, 10, 0, 1, 1, 10)
(1, 10, 1, 10, 0, 1)
(1, 10, 1, 10, 1, 10)


In [444]:
all_focks.shape

(10, 10, 10)

In [459]:
sum(sum(sum(all_focks * measurement_matrices[0])))

<tf.Tensor: shape=(), dtype=float32, numpy=0.71177024>

In [460]:
result.state.fock_prob([0,0,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.71177024>

In [313]:
sum(sum(sum(all_focks * measurement_matrices[1])))


0.15747141559651928

In [323]:
result.state.fock_prob([0, 0, 1]) + result.state.fock_prob([0, 0, 2])


0.15747141559651928

In [321]:
sum(sum(sum(all_focks * measurement_matrices[2])))


0.15747141559651937

In [316]:
result.state.fock_prob([0,1,0])+result.state.fock_prob([0,2,0])

0.15747141559651937

In [324]:
sum(sum(sum(all_focks * measurement_matrices[3])))


0.03499364791033762

In [325]:
result.state.fock_prob([0,1,1])+result.state.fock_prob([0,1,2])+result.state.fock_prob([0,2,1])+result.state.fock_prob([0,2,2])

0.03499364791033762

In [326]:
sum(sum(sum(all_focks * measurement_matrices[4])))


0.04658354409824146

In [327]:
result.state.fock_prob([1,0,0])+result.state.fock_prob([2,0,0])

0.04658354409824146

In [328]:
sum(sum(sum(all_focks * measurement_matrices[5])))


0.012048320061742942

In [329]:
result.state.fock_prob([1,0,1])+result.state.fock_prob([2,0,1])+result.state.fock_prob([1,0,2])+result.state.fock_prob([2,0,1])

0.012802915999067759

In [330]:
sum(sum(sum(all_focks * measurement_matrices[6])))


0.012048320061742945

In [331]:
result.state.fock_prob([1,1,0])+result.state.fock_prob([2,1,0])+result.state.fock_prob([1,2,0])+result.state.fock_prob([2,2,0])

0.012048320061742944

In [332]:
sum(sum(sum(all_focks * measurement_matrices[7])))


0.0001574855880239232

In [333]:
result.state.fock_prob([1,1,1])+result.state.fock_prob([1,1,2])+result.state.fock_prob([1,2,1])+result.state.fock_prob([1,2,2])+result.state.fock_prob([2,1,1])+result.state.fock_prob([2,1,2])+result.state.fock_prob([2,2,1])+result.state.fock_prob([2,2,2])

0.0001574855880239232

In [45]:
result.state.dm().shape

(5, 5, 5, 5)

In [334]:
import tensorflow as tf
import os

In [335]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [341]:
measurement_matrices[1]

array([[[0., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)

In [343]:
tf.linalg.LinearOperatorFullMatrix(measurement_matrices[1]).to_dense()

<tf.Tensor: shape=(3, 3, 3), dtype=float32, numpy=
array([[[0., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)>

In [455]:
eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": cutoff_dim})

In [456]:
result = eng.run(prog, args={
    "alpha": alphas[0],
    "beta": betas[0],
})

In [457]:
all_focks = result.state.all_fock_probs()
measurement_matrices = generate_measurement_matrices(num_modes=modes, cutoff_dim=cutoff_dim)

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
(0, 1, 0, 1, 0, 1)
(0, 1, 0, 1, 1, 3)
(0, 1, 1, 3, 0, 1)
(0, 1, 1, 3, 1, 3)
(1, 3, 0, 1, 0, 1)
(1, 3, 0, 1, 1, 3)
(1, 3, 1, 3, 0, 1)
(1, 3, 1, 3, 1, 3)


In [458]:
all_focks

<tf.Tensor: shape=(3, 3, 3), dtype=float32, numpy=
array([[[7.1177024e-01, 1.1388322e-01, 9.1106584e-03],
        [1.1388322e-01, 1.8221321e-02, 0.0000000e+00],
        [9.1106584e-03, 0.0000000e+00, 0.0000000e+00]],

       [[1.4235402e-02, 2.2776648e-03, 1.0009268e-05],
        [2.2776648e-03, 2.0018539e-05, 0.0000000e+00],
        [1.0009268e-05, 0.0000000e+00, 0.0000000e+00]],

       [[1.4235404e-04, 3.2029662e-04, 5.6302138e-06],
        [3.2029662e-04, 1.1260429e-05, 0.0000000e+00],
        [5.6302138e-06, 0.0000000e+00, 0.0000000e+00]]], dtype=float32)>

In [468]:
measurement_op = tf.linalg.LinearOperatorFullMatrix(measurement_matrices[0])
all_focks_op = tf.linalg.LinearOperatorFullMatrix(all_focks)

In [471]:
mult_op = tf.linalg.LinearOperatorKronecker([all_focks_op, measurement_op])


In [472]:
tf.reduce_sum(mult_op.to_dense())

<tf.Tensor: shape=(), dtype=float32, numpy=0.97597927>

In [491]:
tf.reduce_sum(tf.math.multiply(measurement_matrices[3], all_focks))


<tf.Tensor: shape=(), dtype=float32, numpy=0.01822132>

In [484]:
tf.reduce_sum(measurement_matrices[0] * all_focks)


<tf.Tensor: shape=(), dtype=float32, numpy=0.71177024>

In [488]:
tf.reduce_sum(measurement_matrices[3] * all_focks)

<tf.Tensor: shape=(), dtype=float32, numpy=0.01822132>

In [498]:
[tf.reduce_sum(tf.math.multiply(measurement_matrix, all_focks)) for measurement_matrix in measurement_matrices]


[<tf.Tensor: shape=(), dtype=float32, numpy=0.71177024>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.12299388>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.12299388>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.01822132>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.014377756>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0026136008>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0026136008>,
 <tf.Tensor: shape=(), dtype=float32, numpy=3.1278967e-05>]

In [462]:
all_focks

<tf.Tensor: shape=(3, 3, 3), dtype=float32, numpy=
array([[[7.1177024e-01, 1.1388322e-01, 9.1106584e-03],
        [1.1388322e-01, 1.8221321e-02, 0.0000000e+00],
        [9.1106584e-03, 0.0000000e+00, 0.0000000e+00]],

       [[1.4235402e-02, 2.2776648e-03, 1.0009268e-05],
        [2.2776648e-03, 2.0018539e-05, 0.0000000e+00],
        [1.0009268e-05, 0.0000000e+00, 0.0000000e+00]],

       [[1.4235404e-04, 3.2029662e-04, 5.6302138e-06],
        [3.2029662e-04, 1.1260429e-05, 0.0000000e+00],
        [5.6302138e-06, 0.0000000e+00, 0.0000000e+00]]], dtype=float32)>